In [2]:
import requests
import geopandas as gpd
import folium

In [3]:
# Parámetros para la API
api_key = "ad6e06e230f7307598923265cf881c17"
cities = ["Bogotá", "Medellín", "Cali", "Barranquilla"]  # Ciudades objetivo
base_url = "http://api.openweathermap.org/data/2.5/weather"

# Obtener datos para una ciudad
params = {"q": "Bogotá", "appid": api_key, "units": "metric"}
response = requests.get(base_url, params=params)
data = response.json()

print(data)  # Explorar los datos obtenidos

{'coord': {'lon': -74.0817, 'lat': 4.6097}, 'weather': [{'id': 701, 'main': 'Mist', 'description': 'mist', 'icon': '50d'}], 'base': 'stations', 'main': {'temp': 10.32, 'feels_like': 10.02, 'temp_min': 9.7, 'temp_max': 10.93, 'pressure': 1019, 'humidity': 100, 'sea_level': 1019, 'grnd_level': 738}, 'visibility': 5000, 'wind': {'speed': 1.03, 'deg': 0}, 'clouds': {'all': 75}, 'dt': 1732103193, 'sys': {'type': 1, 'id': 8582, 'country': 'CO', 'sunrise': 1732099522, 'sunset': 1732142348}, 'timezone': -18000, 'id': 3688689, 'name': 'Bogota', 'cod': 200}


In [15]:
def get_city_weather(city, api_key):
    params = {"q": city, "appid": api_key, "units": "metric"}
    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        data = response.json()
        return {
            "city": data["name"],
            "lat": data["coord"]["lat"],
            "lon": data["coord"]["lon"],
            "temp": data["main"]["temp"],
            "humidity": data["main"]["humidity"],
            "weather": data["weather"][0]["description"].capitalize(),
            "wind_speed": data["wind"]["speed"],
            "icon": data["weather"][0]["icon"]
        }
    else:
        print(f"Error al obtener datos para {city}")
        return None

In [16]:
weather_data = [get_city_weather(city, api_key) for city in cities if get_city_weather(city, api_key)]
print(weather_data)

[{'city': 'Bogota', 'lat': 4.6097, 'lon': -74.0817, 'temp': 10.32, 'humidity': 100, 'weather': 'Mist', 'wind_speed': 1.03, 'icon': '50d'}, {'city': 'Medellín', 'lat': 6.2518, 'lon': -75.5636, 'temp': 15.93, 'humidity': 99, 'weather': 'Mist', 'wind_speed': 4.92, 'icon': '50d'}, {'city': 'Santiago de Cali', 'lat': 3.4372, 'lon': -76.5225, 'temp': 18.58, 'humidity': 97, 'weather': 'Mist', 'wind_speed': 2.06, 'icon': '50d'}, {'city': 'Barranquilla', 'lat': 10.9639, 'lon': -74.7964, 'temp': 21.88, 'humidity': 100, 'weather': 'Few clouds', 'wind_speed': 2.06, 'icon': '02d'}]


In [32]:
# Coordenadas aproximadas de Colombia
center_coords = [4.570868, -74.297333]
map_base = folium.Map(location=center_coords, zoom_start=6)

In [33]:
for city in weather_data:
    folium.Marker(
        location=[city["lat"], city["lon"]],
        popup=f"{city['city']}: {city['temp']} °C",
        icon=folium.Icon(color="blue", icon="info-sign")
    ).add_to(map_base)

In [19]:
map_base.save("mapa_clima_inicial.html")

In [34]:
def update_map(api_key, cities):
    map_base = folium.Map(location=center_coords, zoom_start=6)
    weather_data = [get_city_weather(city, api_key) for city in cities if get_city_weather(city, api_key)]
    for city in weather_data:
        folium.Marker(
            location=[city["lat"], city["lon"]],
            popup=f"{city['city']}: {city['temp']} °C",
            icon=folium.Icon(color="blue", icon="info-sign")
        ).add_to(map_base)
    map_base.save("mapa_clima_actualizado.html")
    print("Mapa actualizado")

In [35]:
update_map(api_key, cities)

Mapa actualizado


In [36]:
# Añadir capas de fondo con atribuciones
folium.TileLayer(
    'Stamen Terrain',
    name='Terreno',
    attr='Map tiles by Stamen Design, CC BY 3.0 — Map data © OpenStreetMap'
).add_to(map_base)

folium.TileLayer(
    'Stamen Toner',
    name='Toner',
    attr='Map tiles by Stamen Design, CC BY 3.0 — Map data © OpenStreetMap'
).add_to(map_base)

# Añadir control de capas
folium.LayerControl().add_to(map_base)

In [37]:
def get_color(temp):
    if temp < 10:
        return "blue"
    elif 10 <= temp <= 20:
        return "green"
    elif 20 < temp <= 30:
        return "orange"
    else:
        return "red"

for city in weather_data:
    popup_content = f"""
    <h4>{city['city']}</h4>
    <b>Temperatura:</b> {city['temp']} °C<br>
    <b>Humedad:</b> {city['humidity']}%<br>
    <b>Clima:</b> {city['weather']}<br>
    <b>Viento:</b> {city['wind_speed']} m/s<br>
    <img src="http://openweathermap.org/img/wn/{city['icon']}@2x.png" width="50" height="50">
    """
    folium.Marker(
        location=[city["lat"], city["lon"]],
        popup=folium.Popup(popup_content, max_width=250),
        icon=folium.Icon(color=get_color(city["temp"]), icon="info-sign")
    ).add_to(map_base)

In [38]:
for city in weather_data:
    folium.CircleMarker(
        location=[city["lat"], city["lon"]],
        radius=city["temp"] / 2,  # Escalar el tamaño del círculo con la temperatura
        color=get_color(city["temp"]),
        fill=True,
        fill_opacity=0.6,
        popup=f"Temperatura: {city['temp']} °C"
    ).add_to(map_base)

In [39]:
map_base.save("mapa_clima_final.html")